In [2]:
import os
from glob import glob

import cv2
import numpy as np
from skimage.measure import label, regionprops

In [3]:
video_base_path = "/gpfs/soma_fs/nfc/nfc3008/Videos/"
experiment_folder = "20250430_155345"

video_files = glob(os.path.join(video_base_path, experiment_folder, "*.mp4"))

print(f"Found {len(video_files)} video files.")

Found 969 video files.


In [30]:
def resize(frame, f=0.3):
    return cv2.resize(frame, (0, 0), fx=f, fy=f)


In [36]:
video_file = cv2.VideoCapture(video_files[10])

# Create a background subtractor object
bgs = cv2.createBackgroundSubtractorMOG2(history=100, detectShadows=True)
bgs.setShadowThreshold(0.75)

while True:
    ret, frame = video_file.read()

    if not ret:
        break

    # Apply the background subtractor to get the foreground and shadow mask
    fg_mask = bgs.apply(frame)
    shadow_mask = cv2.inRange(fg_mask, 127, 127)
    fly_mask = cv2.inRange(fg_mask, 255, 255)

    # contours
    fly_contours, _ = cv2.findContours(fly_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    shadow_contours, _ = cv2.findContours(shadow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # find largest contour
    fly_contours = sorted(fly_contours, key=cv2.contourArea, reverse=True)[:1]
    shadow_contours = sorted(shadow_contours, key=cv2.contourArea, reverse=True)[:1]

    # draw largest contour for each
    if len(fly_contours) > 0:
        fly_contour = fly_contours[0]
        if cv2.contourArea(fly_contour) < 100:
            continue
        cv2.drawContours(frame, [fly_contour], -1, (0, 255, 0), 3)
    
    if len(shadow_contours) > 0:
        shadow_contour = shadow_contours[0]
        if cv2.contourArea(shadow_contour) < 100:
            continue
        cv2.drawContours(frame, [shadow_contour], -1, (255, 0, 0), 3)

    frame = resize(frame)


    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_file.release()
cv2.destroyAllWindows()